# download libs
pip install torch transformers accelerate bitsandbytes langchain langchain-community langchain-experimental langchain-huggingface langchain-chroma langchain-text-splitters langchain-core chromadb

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from transformers import BitsAndBytesConfig # for compressing model e.g. 16bits -> 4bits

from transformers import (
                          AutoTokenizer, # Tokenize Model
                          AutoModelForCausalLM,  # LLM Loader - used for loading and using pre-trained models designed for causal language modeling tasks
                          pipeline) # pipline to setup llm-task oritented model
                                    # pipline("text-classification", model='model', device=0)

from langchain_huggingface import HuggingFaceEmbeddings # huggingface sentence_transformer embedding models
from langchain_huggingface.llms import HuggingFacePipeline # like transformer pipeline

from langchain.memory import ConversationBufferMemory # Deprecated
from langchain_community.chat_message_histories import ChatMessageHistory # Deprecated
from langchain_community.document_loaders import PyPDFLoader, TextLoader # PDF Processing
from langchain.chains import ConversationalRetrievalChain # Deprecated
from langchain_experimental.text_splitter import SemanticChunker # module for chunking text

from langchain_chroma import Chroma # AI-native vector databases (ai-native mean built for handle large-scale AI workloads efficiently)
from langchain_text_splitters import RecursiveCharacterTextSplitter # recursively divide text, then merge them together if merge_size < chunk_size
from langchain_core.runnables import RunnablePassthrough # Use for testing (make 'example' easy to execute and experiment with)
from langchain_core.output_parsers import StrOutputParser # format LLM's output text into (list, dict or any custom structure we can work with)
from langchain import hub

In [ ]:
# Read PDF file
Loader = PyPDFLoader
# FILE_PATH = "25 Thuật Ngữ AI - Machine Learning dễ hiểu cho người mới.pdf"
FILE_PATH = "iot_security_report.pdf"
loader = Loader(FILE_PATH)
documents = loader.load()

In [ ]:
print(documents[:50])

[bkai-foundation-model 2024](https://huggingface.co/bkai-foundation-models/vietnamese-bi-encoder)

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name = "bkai-foundation-models/vietnamese-bi-encoder",
    model_kwargs = {'device': 'cuda'},
    encode_kwargs = {'normalize_embeddings': True}
) # convert text to vector (not chunking yet)

In [ ]:
# runtime:
# + bkai-foundation-models/vietnamese-bi-encoder: 3 mins
# + keepitreal/vietnamese-sbert: 3mins
semantic_splitter = SemanticChunker(
    embeddings=embeddings,
    buffer_size=1, # total sentence collected before perform text split
    breakpoint_threshold_type='percentile', # set splitting style: 'percentage' of similarity
    breakpoint_threshold_amount=95, # split text if similarity score > 95%
    min_chunk_size=500,
    add_start_index=True, # assign index for chunk
)

docs = semantic_splitter.split_documents(documents)
print("Number of sementic chunks:", len(docs))

In [ ]:
vector_db = Chroma.from_documents(documents=docs,
                                  embedding=embeddings)

retriever = vector_db.as_retriever()

In [ ]:
result = retriever.invoke("IoT là gì ?")
print("Num of relevant documents: ", len(result))

#? Không Embedd được hình (ý nghĩa của hình)
#? May retrieve duplicate documents
for i, doc in enumerate(result, 1):
    print(f"\n📄 Documellmnt {i}")
    print("-" * 60)
    print(f"📄 Page       : {doc.metadata.get('page_label', doc.metadata.get('page'))}")
    print(f"📝 Content    :\n{doc.page_content.strip()}")
    print("-" * 60)

In [ ]:
with open('token.txt', 'r') as f:
    hg_token = f.read() #? read huggingface token from token.txt file

In [ ]:
# set up config
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#? Initialize Model and Tokenizer
#? PhoGPT-5.5B
#? Phi-2 (2.7B)
#? lmsys/vicuna-7b-v1.5
MODEL_NAME= "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config, # add config
    low_cpu_mem_usage=True,
    token=hg_token
).to("cuda")

In [ ]:
# Check if the model is on CUDA
if next(model.parameters()).is_cuda:
    print("Model is running on CUDA.")
else:
    print("Model is not running on CUDA.")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=True,
    padding_side='left',   # 'left' or 'right' depending on model style (e.g., causal LM often prefers left)
    truncation_side='left'
)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

# #? Integrated tokenizer and model into a Pipeline (for convinient)
model_pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024, # output token
    device_map="auto" # auto allocate GPU if available
)

llm = HuggingFacePipeline(
    pipeline=model_pipeline,
)

## Learn how to prompt so the LLM can generate better multiple-choice question

Ví dụ về một câu hỏi trắc nghiệm tốt:

Câu hỏi: Tấn công side-channel là gì?

Phương án:

A. Là tấn công từ xa vào giao diện web.

B. Là kiểu tấn công dựa trên hành vi tiêu thụ năng lượng của thiết bị.

C. Là tấn công trực diện vào hạ tầng mạng

D. Là tấn công dựa vào bức xạ điện từ để lấy khóa mã hóa.

Đáp án đúng: D

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

parser = StrOutputParser()


In [ ]:
import os

def save_result(result, file_path):
    with open(file_path, 'w') as f:
            f.write(result)

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt = PromptTemplate.from_template("""
        Trả lời ngắn gọn, rõ ràng bằng tiếng việt và chỉ dựa trên thông tin có sẵn bên dưới.
        Nếu không tìm thấy thông tin, hãy nói rõ là không có dữ liệu liên quan.

        Nội dung tài liệu:
        {context}

        Câu hỏi:
        {question}

        Trả lời:
""") #? dùng {{ }} để langchain không nhận string bên trong {} là Biến

rag_chain = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

query = 'Liệt kê các thành phần trong hệ thống IoT ?'
result = rag_chain.invoke(query)

In [ ]:
print(result)

### Customize RAG Output to Json

In [ ]:
multi_choice_prompt = """
        Dựa vào nội dung sau, hãy:
        1. Tóm tắt tối đa 3 ý chính, kèm theo số trang nếu có.
        2. Trả lời câu hỏi bằng tiếng Việt ngắn gọn và chính xác.
        3. Nếu không có thông tin liên quan, hãy để "Answer" là "Không có dữ liệu liên quan".

        Đảm bảo trả kết quả **ở dạng JSON** với cấu trúc sau:
        {{"main_ideas": [
            {{"point": "Ý chính 1", "source": "Trang ..."}},
            {{"point": "Ý chính 2", "source": "Trang ..."}},
            {{"point": "Ý chính 3", "source": "Trang ..."}}
        ],
        "answer": "Câu trả lời ngắn gọn"
        }}

        Vui lòng chỉ trả lời bằng format JSON, không giải thích thêm.

        Context:
        {context}

        Question:
        {question}

        Answer:

""" #? dùng {{ }} để langchain không nhận string bên trong {} là Biến

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

class MainIdea(BaseModel):
    point: str
    source: str

class QAResponse(BaseModel):
    main_ideas: List[MainIdea]
    answer: str

parser = PydanticOutputParser(pydantic_object=QAResponse)
prompt_template = PromptTemplate(
    template=multi_choice_prompt,
    input_variables=["context", "question"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | parser
)

# query = 'Liệt kê các thành phần trong hệ thống IoT ?'
query = 'IoT là gì ?'
result = rag_chain.invoke(query)
print(result)

In [ ]:
multi_choice_prompt = """
        Dựa vào nội dung sau, hãy:
        1. Tóm tắt tối đa 3 ý chính, kèm theo số trang nếu có.
        2. Trả lời câu hỏi bằng tiếng Việt ngắn gọn và chính xác.
        3. Nếu không có thông tin liên quan, hãy để "Answer" là "Không có dữ liệu liên quan".

        Đảm bảo trả kết quả **ở dạng JSON** với cấu trúc sau:
        {{"main_ideas": [
            {{"point": "Ý chính 1", "source": "Trang ..."}},
            {{"point": "Ý chính 2", "source": "Trang ..."}},
            {{"point": "Ý chính 3", "source": "Trang ..."}}
        ],
        "answer": "Câu trả lời ngắn gọn"
        }}

        Vui lòng chỉ trả lời bằng format JSON, không giải thích thêm.

        Context:
        {context}

        Question:
        {question}

        Answer:

""" #? dùng {{ }} để langchain không nhận string bên trong {} là Biến

In [ ]:
def run_custom_rag(user_question):
    prompt = PromptTemplate.from_template(multi_choice_prompt)
    rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | parser
    )


    query = user_question
    result = rag_chain.invoke(query)

    file_path = 'output.txt'
    save_result(result, file_path)


    return result

In [ ]:
question = "IoT là gi?"
result = run_custom_rag(question)

In [ ]:
query = "Các thành phần trong hệ thống IoT bao gồm những gì ?"
result = run_custom_rag(query)